In [55]:
import pandas as pd
import zipfile
import os
import json
import re
# import fasttext
import multiprocessing
from statistics import mean, median, mode
import datetime
import numpy as np

In [84]:
test = pd.read_csv('SOTAB CTA/sotab_v2_cta_test_set.csv')

In [85]:
# split by grouping tables and counting labels
grouped_by_table = test.groupby(["table_name"])["label"].apply(','.join).reset_index()
grouped_by_table

,table_name,label
0,Book_1jour-1jeu.com_September2020_CTA.json.gz,"Number,Date,Book/name,BookFormatType"
1,Book_abebooks.co.uk_September2020_CTA.json.gz,"Book/name,BookFormatType"
2,Book_abebooks.de_September2020_CTA.json.gz,"BookFormatType,Book/name"
3,Book_andy-palmer.co.uk_September2020_CTA.json.gz,"Person/name,IdentifierAT,BookFormatType,Language"
4,Book_apgaming.co.uk_September2020_CTA.json.gz,"Review,Language,IdentifierAT,Book/name,BookFor..."
...,...,...
604,TVEpisode_go-stream.fun_September2020_CTA.json.gz,"TVEpisode/name,CreativeWorkSeries"
605,TVEpisode_gostream.plus_September2020_CTA.json.gz,"TVEpisode/name,CreativeWorkSeries"
606,TVEpisode_gostream123.pw_September2020_CTA.jso...,"TVEpisode/name,CreativeWorkSeries"
607,TVEpisode_gslps.org_September2020_CTA.json.gz,"TVEpisode/name,CreativeWorkSeries"


In [86]:
X = grouped_by_table[["table_name"]].values
y = np.zeros(shape=(len(grouped_by_table["table_name"].tolist()), len(test["label"].unique()))) #encoded labels
for index, row in grouped_by_table.iterrows():
    table_labels = row["label"].split(',')
    count = 0
    
    for label in test["label"].unique():
        if label in table_labels:
            y[index][count] = 1
        else:
            y[index][count] = 0
        count += 1

In [5]:
# ! pip install scikit-learn
# ! pip install scikit-multilearn


In [154]:
from collections import Counter
from skmultilearn.model_selection import iterative_train_test_split
from skmultilearn.model_selection.measures import get_combination_wise_output_matrix

X_train, y_train, X_test, y_test = iterative_train_test_split(X, y, test_size = 0.15)
print('Training set length: '+str(len(X_train)) +', Testing set length: '+ str(len(X_test)))

Training set length: 499, Testing set length: 110


In [155]:
# Labels in each set
p = pd.DataFrame({
    'train': Counter(str(combination) for row in get_combination_wise_output_matrix(y_train, order=1) for combination in row),
    'test' : Counter(str(combination) for row in get_combination_wise_output_matrix(y_test, order=1) for combination in row)
}).T.fillna(0.0)
p

,"(1,)","(44,)","(2,)","(54,)","(27,)","(17,)","(28,)","(81,)","(9,)","(0,)",...,"(68,)","(59,)","(40,)","(39,)","(38,)","(60,)","(62,)","(61,)","(79,)","(63,)"
train,26,18,53,24,19,26,18,10,38,37,...,9,8,4,10,8,17,9,6,8,8
test,6,5,9,4,5,5,3,1,6,6,...,2,2,1,2,3,4,2,1,2,2


In [156]:
#training_tables = [ table[0] for table in X_train ]
sample_testing_tables = [ table[0] for table in X_test ]

In [157]:
test_set = test[test["table_name"].isin(sample_testing_tables)]
test_set

,table_name,column_index,label
5,Event_blugrottonj.com_September2020_CTA.json.gz,4,Place/name
6,Event_blugrottonj.com_September2020_CTA.json.gz,5,telephone
29,LocalBusiness_basellive.ch_September2020_CTA.j...,2,openingHours
32,LocalBusiness_worldcement.com_September2020_CT...,5,Country
34,Movie_321movies.org_September2020_CTA.json.gz,10,Rating
...,...,...,...
1824,Product_babyartikelcheck.de_September2020_CTA....,0,Product/name
1832,Product_divas-club.de_September2020_CTA.json.gz,0,Product/name
1833,Product_divas-club.de_September2020_CTA.json.gz,1,Product/description
1846,Person_crsdenver.com_September2020_CTA.json.gz,3,faxNumber


In [158]:
len(test_set["label"].unique())

82

In [159]:
test_set.to_csv('SOTAB CTA/sample_test.csv', index=False)